In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import statistics
from statistics import mean
from mpl_toolkits import mplot3d

%matplotlib inline

In [40]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [41]:
ccCustomersCSv = pd.read_csv('Resources/BankChurners.csv')

In [42]:
ccCustomersCSv

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.999910
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.999940
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.999980
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.999870
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0.000191,0.999810
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,0.995270,0.004729
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.997880,0.002118
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.996710,0.003294


In [43]:
ccCustomersCSv.index

RangeIndex(start=0, stop=10127, step=1)

In [44]:
# some rows have the value 'Unknown' in the income category column, filtering those out
ccCustomersCSv = ccCustomersCSv[ccCustomersCSv['Income_Category'].str.contains('Unknown')== False]

In [45]:
ccCustomersCSv['Income_Category'].unique()

array(['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
       '$120K +'], dtype=object)

In [46]:
ccCustomersCSv['Credit_Limit'].max()

34516.0

In [47]:
ccCustomersCSv['Credit_Limit'].min()

1438.3

In [48]:
ccCustomersCSv['Credit_Limit'].median()

4331.0

In [49]:
ccCustomersCSv.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
                             'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1'], axis=1, inplace=True)


/var/folders/2w/fy2bjdfs7nz5_675nfk08_jh0000gp/T/ipykernel_69499/934961141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccCustomersCSv.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',


In [50]:
df = ccCustomersCSv.copy()

In [51]:
df['Credit_Limit']=df['Credit_Limit'].astype(float)

In [52]:
def new_CreditLimit(sample):
    if sample >= 4549.0:
        return 1
    else:
        return 0

In [53]:
df['Credit_Limit'] = df['Credit_Limit'].apply(new_CreditLimit)

In [54]:
df['Credit_Limit'].value_counts()

0    4633
1    4382
Name: Credit_Limit, dtype: int64

In [55]:
X_dummies = pd.get_dummies(df)
print(X_dummies.columns)
X_dummies

Index(['CLIENTNUM', 'Customer_Age', 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Attrition_Flag_Attrited Customer', 'Attrition_Flag_Existing Customer',
       'Gender_F', 'Gender_M', 'Education_Level_College',
       'Education_Level_Doctorate', 'Education_Level_Graduate',
       'Education_Level_High School', 'Education_Level_Post-Graduate',
       'Education_Level_Uneducated', 'Education_Level_Unknown',
       'Marital_Status_Divorced', 'Marital_Status_Married',
       'Marital_Status_Single', 'Marital_Status_Unknown',
       'Income_Category_$120K +', 'Income_Category_$40K - $60K',
       'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K',
       'Income_Category_Less than $40K', 'Card_Category_Blue',
 

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,...,Marital_Status_Unknown,Income_Category_$120K +,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,768805383,45,3,39,5,1,3,1,777,11914.0,...,0,0,0,1,0,0,1,0,0,0
1,818770008,49,5,44,6,1,2,1,864,7392.0,...,0,0,0,0,0,1,1,0,0,0
2,713982108,51,3,36,4,1,0,0,0,3418.0,...,0,0,0,0,1,0,1,0,0,0
3,769911858,40,4,34,3,4,1,0,2517,796.0,...,1,0,0,0,0,1,1,0,0,0
4,709106358,40,3,21,5,1,0,1,0,4716.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,50,2,40,3,2,3,0,1851,2152.0,...,0,0,1,0,0,0,1,0,0,0
10123,710638233,41,2,25,4,2,3,0,2186,2091.0,...,0,0,1,0,0,0,1,0,0,0
10124,716506083,44,1,36,5,3,4,1,0,5409.0,...,0,0,0,0,0,1,1,0,0,0
10125,717406983,30,2,36,4,3,3,1,0,5281.0,...,1,0,1,0,0,0,1,0,0,0


In [56]:
X = X_dummies.drop('Credit_Limit', axis=1)
y = X_dummies['Credit_Limit']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [57]:
clf = RandomForestClassifier(random_state=42, n_estimators=1000)

In [58]:
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [59]:
print(f"Training Data Score random forests classifier: {clf.score(X_train, y_train)}")
print(f"Testing Data Score random forests classifier: {clf.score(X_test, y_test)}")

Training Data Score random forests classifier: 1.0
Testing Data Score random forests classifier: 0.9933451641526175


In [60]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
confusion_matrix(y_true, y_pred)


array([[1140,    3],
       [  12, 1099]])